# Knowledge Graph Agent with LlamaParse

<a href="https://colab.research.google.com/github/run-llama/llama_parse/blob/main/examples/knowledge_graphs/kg_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here we build a knowledge graph agent over the SF 2023 Budget Proposal. We use LlamaIndex abstractions to construct a knowledge graph, and we store the property graph in neo4j. We then build an agent that can interact with the knowledge graph as a tool.

## Setup (Installs, Data, Models)

In [ ]:
!pip install llama-index
!pip install llama-index-core==0.10.42
!pip install llama-index-embeddings-openai
!pip install llama-index-postprocessor-flag-embedding-reranker
!pip install git+https://github.com/FlagOpen/FlagEmbedding.git
!pip install llama-index-graph-stores-neo4j
!pip install llama-parse

In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
import os

# API access to llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-urdbonhLECN2x1yLa0WJZXTNE333PFV2o2icaigJ5sScwnrR"

#### Setup Model

Here we use gpt-4o and default OpenAI embeddings.

In [11]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from Setting import openAIKey

llm = OpenAI(model="gpt-4o",api_key=openAIKey.Key)
embed_model = OpenAIEmbedding(api_key=openAIKey.Key,model="text-embedding-3-small")

Settings.llm = llm
Settings.embed_model = embed_model

#### Load Data

Here we load the 2023 Budget PDF and parse it with LlamaParse.

In [4]:
!mkdir data
!wget "https://www.dropbox.com/scl/fi/vip161t63s56vd94neqlt/2023-CSF_Proposed_Budget_Book_June_2023_Master_Web.pdf?rlkey=hemoce3w1jsuf6s2bz87g549i&dl=0" -O data/budget_2023.pdf

mkdir: data: File exists
--2024-07-07 11:13:29--  https://www.dropbox.com/scl/fi/vip161t63s56vd94neqlt/2023-CSF_Proposed_Budget_Book_June_2023_Master_Web.pdf?rlkey=hemoce3w1jsuf6s2bz87g549i&dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf22ab5d2f7b764381590a2e7d3.dl.dropboxusercontent.com/cd/0/inline/CWOfOTFpUnn79XOXjvldZCYs5AXvrufdL94813TJw5JIAYHb0YvMxewYMVuxWQjLvqo7H1zsGqRDpGtLpfDjymTiitbAlo3_GunH9GDDSRcFccOUz5JJHiAuEQLoC4vjj8I/file# [following]
--2024-07-07 11:13:30--  https://ucf22ab5d2f7b764381590a2e7d3.dl.dropboxusercontent.com/cd/0/inline/CWOfOTFpUnn79XOXjvldZCYs5AXvrufdL94813TJw5JIAYHb0YvMxewYMVuxWQjLvqo7H1zsGqRDpGtLpfDjymTiitbAlo3_GunH9GDDSRcFccOUz5JJHiAuEQLoC4vjj8I/file
Resolving ucf22ab5d2f7b764381590a2e7d3.dl.dropboxusercontent.com (ucf22ab5d2f7b764381590a2e7d3.dl.dropboxusercontent.com)... 162.125.1.15
Conne

In [5]:
from llama_parse import LlamaParse

docs = LlamaParse(result_type="text").load_data("./data/budget_2023.pdf")

Started parsing the file under job_id cac11eca-59d5-4f94-b374-271a3413d914


In [6]:
from copy import deepcopy
from llama_index.core.schema import TextNode, Document
from llama_index.core import VectorStoreIndex


def get_sub_docs(docs):
    """Split docs into pages, by separator."""
    sub_docs = []
    for doc in docs:
        doc_chunks = doc.text.split("\n---\n")
        for doc_chunk in doc_chunks:
            sub_doc = Document(
                text=doc_chunk,
                metadata=deepcopy(doc.metadata),
            )
            sub_docs.append(sub_doc)

    return sub_docs

In [7]:
# this will split into pages
sub_docs = get_sub_docs(docs)

#### Initialize Graph Store

Here we use Neo4j but you can also use our other integrations like Nebula (see an [example notebook](https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/property_graph/property_graph_advanced.ipynb)).

To launch Neo4j locally, first ensure you have docker installed. Then, you can launch the database with the following docker command

```bash
docker run \
    -p 7474:7474 -p 7687:7687 \
    -v $PWD/data:/data -v $PWD/plugins:/plugins \
    --name neo4j-apoc \
    -e NEO4J_apoc_export_file_enabled=true \
    -e NEO4J_apoc_import_file_enabled=true \
    -e NEO4J_apoc_import_file_use__neo4j__config=true \
    -e NEO4JLABS_PLUGINS=\[\"apoc\"\] \
    neo4j:latest
```

From here, you can open the db at [http://localhost:7474/](http://localhost:7474/). On this page, you will be asked to sign in. Use the default username/password of `neo4j` and `neo4j`.

Once you login for the first time, you will be asked to change the password.

After this, you are ready to create your first property graph!

In [8]:
from llama_index.graph_stores.neo4j import Neo4jPGStore

graph_store = Neo4jPGStore(
    username="neo4j",
    password="Th503221.",
    url="bolt://localhost:7687",
)
vec_store = None

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


## Construct Knowledge Graph, Get Retrievers

This section shows you how to construct the knowledge graph over the existing documents.

**Note**: we have the default extractors (implicit path, simple llm path) configured. You can also choose to use a pre-defined schema as mentioned in this [notebook](https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/property_graph/property_graph_advanced.ipynb).

In [9]:
from llama_index.core.indices.property_graph import (
    ImplicitPathExtractor,
    SimpleLLMPathExtractor,
)
from llama_index.core import PropertyGraphIndex
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

In [14]:
index = PropertyGraphIndex.from_documents(
    sub_docs,
    embed_model=embed_model,
    kg_extractors=[
        ImplicitPathExtractor(),
        SimpleLLMPathExtractor(
            llm=llm,
            num_workers=4,
            max_paths_per_chunk=10,
        ),
    ],
    property_graph_store=graph_store,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/362 [00:00<?, ?it/s]

Extracting paths from text:  11%|█         | 49/437 [00:41<05:31,  1.17it/s]


KeyboardInterrupt: 

Retrying llama_index.llms.openai.base.OpenAI._achat in 0.5601892574820082 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-ADg9O6qOx3fmDvCTqnDjoryv on tokens per min (TPM): Limit 30000, Used 29771, Requested 1412. Please try again in 2.366s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.24476692769405572 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-ADg9O6qOx3fmDvCTqnDjoryv on tokens per min (TPM): Limit 30000, Used 29706, Requested 1488. Please try again in 2.388s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.022311549632059613 seconds

In [18]:
# run this if index is already loaded
index = PropertyGraphIndex.from_existing(
    graph_store,
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    kg_extractors=[
        ImplicitPathExtractor(),
        SimpleLLMPathExtractor(
            llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3),
            num_workers=4,
            max_paths_per_chunk=10,
        ),
    ],
    show_progress=True,
)

Extracting implicit paths: 0it [00:00, ?it/s]
Extracting paths from text: 0it [00:00, ?it/s]

Extracting paths from text: 0it [00:00, ?it/s]
Generating embeddings: 0it [00:00, ?it/s]
Generating embeddings: 0it [00:00, ?it/s]
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


The constructed knowledge graph should look something like this
![knowledge graph](https://github.com/run-llama/llama_parse/blob/main/examples/knowledge_graphs/sf2023_budget_kg_screenshot.png?raw=1)

#### Define Vector Retriever

Here we define our vector context retriever - it returns initial nodes via vector search, and traverses the relations to pull in more nodes/context.

In [ ]:
from llama_index.core.indices.property_graph import VectorContextRetriever

kg_retriever = VectorContextRetriever(
    index.property_graph_store,
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    similarity_top_k=2,
    path_depth=1,
    # include_text=False,
    include_text=True,
)

In [ ]:
nodes = kg_retriever.retrieve(
    "Give me all the programs that the mayor's budget includes"
)
# nodes = kg_retriever.retrieve('san francisco')
print(len(nodes))
for idx, node in enumerate(nodes):
    print(f">> IDX: {idx}, {node.get_content()}")

3
>> IDX: 0, Here are some facts extracted from the provided text:

Mayor's budget -> Includes -> Key changes

first responders to petition for an individual to enter
the programs. In these procedures, a CARE Plan is
established, and a judge can use court orders with
support such as short-term stabilization medications
and beds, as well as wellness and recovery offerings.
The Mayor’s proposed budget includes funding
for engagement and assessment staff, new City
attorneys dedicated to CARE Court implementation,
increased capacity for treatment and housing, and
outreach and educational efforts.
Improvements at Laguna Honda Hospital
Beyond behavioral health, this budget makes
investments in DPH’s budget for Laguna Honda
Hospital, which is actively working towards gaining
recertification with the Centers for Medicare
and Medicaid Services (CMS). DPH is currently
implementing the action plan submitted to CMS,
and it represents a significant facility-wide effort
and includes hundreds of proc

## Build Baseline Vector Index

We also build a "baseline" vector index. This follows the "naive" RAG pipeline approach of chunking and vector embedding. We use this as a comparison point.

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core.query_engine import RetrieverQueryEngine

base_index = VectorStoreIndex.from_documents(sub_docs, embed_model=embed_model)
base_retriever = base_index.as_retriever(similarity_top_k=2)
base_query_engine = RetrieverQueryEngine(base_retriever)

In [ ]:
response = base_query_engine.query(
    "Give me all the programs that the mayor's budget includes"
)
print(str(response))

The mayor's budget includes the following programs:

1. Ongoing programmatic support for all districts.
2. One-time $5.0 million and $250,000 ongoing investment in priority community-based organization needs, including capital and infrastructure, and public safety.
3. Support for the Mayor’s Office Administration, which advances Mayoral priorities through policy and budget development, communications, and advocacy.
4. Financial Capability Services.
5. Nonprofit Capacity Building.
6. Eviction Prevention and Housing Stabilization Services.
7. Community and Housing Place-Based Services.
8. Civil Legal Services.
9. Supportive Housing for Persons with HIV/AIDS.
10. Community, Coalition, and Cultural District Building.
11. Rental and Homeownership Counseling.
12. Capital Projects.
13. Housing Development Grants.
14. Creation of permanently affordable housing.
15. Foster healthy communities and neighborhoods.
16. Improve access to affordable housing.
17. Preserve affordable housing.
18. Promo

In [ ]:
print(len(response.source_nodes))
for node in response.source_nodes:
    print("---")
    print(node.get_content())

## Build Custom Retriever

Build joint retriever that combines vector and KG search.

In [ ]:
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore
from typing import List


class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both KG vector search and direct vector search."""

    def __init__(self, kg_retriever, vector_retriever):
        self._kg_retriever = kg_retriever
        self._vector_retriever = vector_retriever

    def _retrieve(self, query_bundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""
        kg_nodes = self._kg_retriever.retrieve(query_bundle)
        vector_nodes = self._vector_retriever.retrieve(query_bundle)

        unique_nodes = {n.node_id: n for n in kg_nodes}
        unique_nodes.update({n.node_id: n for n in vector_nodes})
        return list(unique_nodes.values())

In [ ]:
custom_retriever = CustomRetriever(kg_retriever, base_retriever)

In [ ]:
nodes = custom_retriever.retrieve(
    "Give me all the programs that the mayor's budget includes"
)
# len(nodes)

## Build Agent

Now that we have the retriever, we can treat it as a RAG pipeline tool, and wrap it with an agent that can perform basic CoT reasoning and maintain conversation memory over time.

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import RetrieverQueryEngine

kg_query_engine = RetrieverQueryEngine(custom_retriever)
kg_query_tool = QueryEngineTool(
    query_engine=kg_query_engine,
    metadata=ToolMetadata(
        name="query_tool",
        description="Provides information about the 2023 SF Budget Report.",
    ),
)

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker

agent_worker = FunctionCallingAgentWorker.from_tools(
    [kg_query_tool],
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)
agent = agent_worker.as_agent()

## Try out Queries

Now that the agent is setup, let's try out some queries.

In [ ]:
response = agent.chat("Give me all the programs that the mayor's budget includes")

Added user message to memory: Give me all the programs that the mayor's budget includes
=== Calling Function ===
Calling function: query_tool with args: {"input": "all programs included in the mayor's budget"}
=== Function Output ===
The mayor's budget includes a variety of programs and investments:

1. **CARE Court Implementation**: Funding for engagement and assessment staff, new City attorneys, increased capacity for treatment and housing, and outreach and educational efforts.
2. **Laguna Honda Hospital**: Over $3.5 million for staffing in key areas, including education and training, patient care experience, medication management, and leadership.
3. **Economic Recovery**: $24.4 million over two years for the Roadmap for Downtown San Francisco’s Future and economic recovery across the city.
4. **Tax Relief and Incentives**: Changes to business taxes, including delaying tax increases for certain industries and offering discounts on office-based gross receipts tax for new offices.
5. *

In [ ]:
print(str(response))

The mayor's budget includes a comprehensive array of programs and investments, covering various sectors and priorities:

1. **CARE Court Implementation**: Funding for engagement and assessment staff, new City attorneys, increased capacity for treatment and housing, and outreach and educational efforts.
2. **Laguna Honda Hospital**: Over $3.5 million for staffing in key areas, including education and training, patient care experience, medication management, and leadership.
3. **Economic Recovery**: $24.4 million over two years for the Roadmap for Downtown San Francisco’s Future and economic recovery across the city.
4. **Tax Relief and Incentives**: Changes to business taxes, including delaying tax increases for certain industries and offering discounts on office-based gross receipts tax for new offices.
5. **Small Business Support**: $5 million in direct grants to help small businesses stabilize, scale, and adapt.
6. **CalAIM Expansion**: Focus on expanded benefits for people at risk o

In [ ]:
agent.reset()
response = agent.chat(
    "Compare the budget for DPA Police Accountabilty from 2022-2023 to 2023-2024"
)
print(str(response))

Added user message to memory: Compare the budget for DPA Police Accountabilty from 2022-2023 to 2023-2024
=== Calling Function ===
Calling function: query_tool with args: {"input": "DPA Police Accountability budget for 2022-2023"}
=== Function Output ===
The DPA Police Accountability budget for 2022-2023 is $9,776,177.
=== Calling Function ===
Calling function: query_tool with args: {"input": "DPA Police Accountability budget for 2023-2024"}
=== Function Output ===
The budget for the Department of Police Accountability for the fiscal year 2023-2024 is $9,990,353.
=== LLM Response ===
The budget for the Department of Police Accountability (DPA) has increased from $9,776,177 in the fiscal year 2022-2023 to $9,990,353 in the fiscal year 2023-2024. This represents an increase of $214,176.
The budget for the Department of Police Accountability (DPA) has increased from $9,776,177 in the fiscal year 2022-2023 to $9,990,353 in the fiscal year 2023-2024. This represents an increase of $214,176.

In [ ]:
print(str(response.source_nodes[0].get_content()))

Here are some facts extracted from the provided text:

Dpa department of police accountability -> Has -> Total funded positions

ORGANIZATIONAL STRUCTURE: POLICE ACCOUNTABILITY


                                                                       Executive Director


                                   Chief of Staff                                                                           Chief of Investigations


          Audit                     Operations                       Legal                                        Investigations                  Mediation


                                                       Policy                     SB 1421


Department Total Budget Historical Comparison (Mayor's Proposed)                                                                      Budget Year 2023-2024 and 2024-2025


                                           Department Total Budget Historical Comparison
 DPA Department Of Police AccountabilityTOTAL BUDGET – HISTORICAL CO